In [1]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import parser
import re

In [2]:
site = 'https://www.hanbit.co.kr'
url = 'https://www.hanbit.co.kr/store/books/new_book_list.html'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)
req = requests.get(url)
html = req.text
time.sleep(2)

In [5]:
soup = BeautifulSoup(driver.page_source,'html.parser')


In [10]:
lis = soup.select('.new_book_list_wrap>div>li')
len(lis)

20

In [11]:
href = li.find('a')['href']
href

NameError: name 'li' is not defined

In [12]:
sub_url = site+href
sub_url

NameError: name 'href' is not defined

In [13]:
sub_req = requests.get(sub_url)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html,'html.parser')

NameError: name 'sub_url' is not defined

In [14]:
lis = soup.select('.new_book_list_wrap>div>li')
len(lis)

20

In [241]:
li = lis[0]

In [9]:
href = li.find('a')['href']
href

NameError: name 'li' is not defined

In [243]:
sub_url = site+href
sub_url

'https://www.hanbit.co.kr/store/books/look.php?p_code=B6694310162'

In [244]:
itemName = li.select_one('.book_tit').get_text()
itemName

'하루 한 장 초등과학 365'

In [245]:
writer = li.select_one('.book_writer').get_text()
writer

'치바 가즈요시 '

In [246]:
#역자
translator = sub_soup.select_one('.info_list>li:nth-child(2)').get_text().split(':')[1]
translator

' 허영은 , 유우종 (감수) '

In [277]:
translator = sub_soup.select_one('.info_list')
len(translator)==13

True

In [307]:
#출간일
date = sub_soup.select_one('.info_list').get_text().split('출간')[1][:15]
date = re.sub('[^0-9]','',date)
date = int(date)
date

20211102

In [324]:
#페이지
jpage=sub_soup.select_one('.info_list').get_text().split('쪽')[0][-5:]
jpage = re.sub('[^0-9]','',jpage)
jpage = int(jpage)
jpage

96

In [346]:
#정가
price=sub_soup.select_one('.store_payment_area>fieldset>label>p:nth-child(1)').get_text()
price = re.sub('[^0-9]','',price)
price = int(price)
price

30000

In [352]:
#할인가
sale=sub_soup.select_one('.store_payment_area>fieldset>label>p:nth-child(2)>span:nth-child(2)>strong').get_text()
sale = re.sub('[^0-9]','',sale)
sale = int(sale)
sale

27000

In [201]:
#마일리지
Mileage=sub_soup.select_one('.store_payment_area>fieldset>label>p:nth-child(3)>span:nth-child(2)').get_text().split('점')[0]
Mileage = int(Mileage)
Mileage

990

In [21]:
lines=[]

site = 'https://www.hanbit.co.kr'
url = 'https://www.hanbit.co.kr/store/books/new_book_list.html'
driver = webdriver.Chrome('chromedriver.exe')
#driver.get(url)
req = requests.get(url)
html = req.text
#time.sleep(1)
for page in range(1,11):
    url = 'https://www.hanbit.co.kr/store/books/new_book_list.html?page='+str(page)#+'&brand=&cate1=&cate2=&searchKey=&keyWord='
    #driver.get(url)
    #time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.new_book_list_wrap>div>li')

    for li in lis:
        href = li.find('a')['href']
        sub_url = site+href
        sub_req = requests.get(sub_url)
        sub_html = sub_req.text
        sub_soup = BeautifulSoup(sub_html,'html.parser')
        
        writer = li.select_one('.book_writer').get_text()
        itemName = li.select_one('.book_tit').get_text()

        Mileage=sub_soup.select_one('.store_payment_area>fieldset>label>p:nth-child(3)>span:nth-child(2)').get_text().split('점')[0]
        Mileage = re.sub('[^0-9]','',Mileage)
        Mileage=int(Mileage)
        sale=sub_soup.select_one('.store_payment_area>fieldset>label>p:nth-child(2)>span:nth-child(2)>strong').get_text()
        sale = re.sub('[^0-9]','',sale)
        sale = int(sale)
        price=sub_soup.select_one('.store_payment_area>fieldset>label>p:nth-child(1)').get_text()
        price = re.sub('[^0-9]','',price)
        price = int(price)
        jpage=sub_soup.select_one('.info_list').get_text().split('쪽')[0][-5:]
        jpage = re.sub('[^0-9]','',jpage)
        jpage = int(jpage)
        date = sub_soup.select_one('.info_list').get_text().split('출간')[1][:15]
        date = re.sub('[^0-9]','',date)
        date = int(date)
        date
        if len(sub_soup.select_one('.info_list'))==13:
            translator = sub_soup.select_one('.info_list>li:nth-child(2)').get_text().split(':')[1]
            translator = re.sub('[0-9]','',translator)
            pass
        else:
            translator ='--'
            pass
        lines.append([itemName,writer,translator,date,jpage,price,sale,Mileage])

#driver.close()

In [22]:
df = pd.DataFrame(lines, columns=['제목', '저자', '역자', '출간일', '페이지', '정가', '판가','마일리지'])

In [23]:
df.tail(5)

,제목,저자,역자,출간일,페이지,정가,판가,마일리지
195,나도 영문 손글씨 잘 쓰면 소원이 없겠네,윤정희(리제 캘리그라피),--,20211105,240,16800,15120,840
196,구글 BERT의 정석,수다르산 라비찬디란,"전희원 , 정승환 , 김형준",20211103,368,34000,30600,1700
197,1000개 숨은그림찾기 몬스터 : 찾아도 찾아도 끝판왕,김확실,--,20211102,96,8000,7200,400
198,1000개 숨은그림찾기 보물찾기 : 찾아도 찾아도 끝판왕,전진희,--,20211102,96,8000,7200,400
199,1000개 숨은그림찾기 탈것 : 찾아도 찾아도 끝판왕,이한이,--,20211102,96,8000,7200,400


In [334]:
df.to_csv('한빛도서과제.csv', index=False)

In [18]:
from tqdm.notebook import tqdm 
#시간이 오래 걸리는 작업시 진척도 보여줌

In [20]:
""" #강사님 .딕셔너리 사용.
def convert_unit(s):
    return int(s.replace(',','').replace('원','').replace('점',''))
    
lines = []
for page in tqdm(range(1,11)):
    url = f'{base_url}{sub_url}?page={page}'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    lis = soup.select('.sub_book_list')
    
    for li in lis:
        book_url = li.find('a')['href']
        res = requests.get(base_url + book_url)
        book_soup = BeautifulSoup(res.text, 'html.parser')

        info = book_soup.select_one('.store_product_info_box')
        title = info.find('h3').string
        book_lis = info.select('.info_list > li')
        book_dict = {}
        for li in book_lis:
            items = li.get_text().split(':')
            book_dict[items[0].strip()] = items[1].strip()
        author = book_dict['저자']
        translator = book_dict['번역'] if '번역' in book_dict.keys() else '-' 
        date = book_dict['출간']
        page = book_dict['페이지'].split()[0]
        
        payments = book_soup.select_one('.payment_box.curr').find_all('p')
        price = payments[0].select_one('.pbr').get_text().strip()
        sales = payments[1].select_one('.pbr').get_text().split('(')[0]
        mileage = payments[2].select_one('.pbr').get_text().split()[0]

        lines.append([title,author,translator,date,int(page),
                      convert_unit(price),convert_unit(sales),convert_unit(mileage)])
"""

" #강사님 .딕셔너리 사용.\ndef convert_unit(s):\n    return int(s.replace(',','').replace('원','').replace('점',''))\n    \nlines = []\nfor page in tqdm(range(1,11)):\n    url = f'{base_url}{sub_url}?page={page}'\n    res = requests.get(url)\n    soup = BeautifulSoup(res.text, 'html.parser')\n    lis = soup.select('.sub_book_list')\n    \n    for li in lis:\n        book_url = li.find('a')['href']\n        res = requests.get(base_url + book_url)\n        book_soup = BeautifulSoup(res.text, 'html.parser')\n\n        info = book_soup.select_one('.store_product_info_box')\n        title = info.find('h3').string\n        book_lis = info.select('.info_list > li')\n        book_dict = {}\n        for li in book_lis:\n            items = li.get_text().split(':')\n            book_dict[items[0].strip()] = items[1].strip()\n        author = book_dict['저자']\n        translator = book_dict['번역'] if '번역' in book_dict.keys() else '-' \n        date = book_dict['출간']\n        page = book_dict['페이지'].split()[0]